# 🧠 Trenowanie i porównanie modeli YOLO w zadaniu detekcji odpadów

## 1. Cel projektu

Celem projektu było:
- wytrenowanie modelu detekcji obiektów typu **YOLO**,
- analiza wpływu jakości danych treningowych oraz liczby epok na jakość modelu,
- porównanie dwóch modeli:
  - **Model A** – trenowany na gorszej jakości zdjęciach i mniejszej liczbie epok,
  - **Model B** – trenowany na lepszej jakości danych i większej liczbie epok.

Projekt dotyczy zadania **detekcji odpadów (Garbage Detection)** na obrazach.

## 2. Opis wykorzystanych modeli

W projekcie wykorzystano architekturę **YOLO11n**, trenowaną w środowisku Python z użyciem biblioteki **Ultralytics YOLO**.

### 2.1 Model A – model słabiej wytrenowany

Model A:
- trenowany na zbiorze danych o **niższej jakości obrazów**,
- trenowany przez **mniejszą liczbę epok**,
- charakteryzuje się:
  - niższym recall,
  - większą liczbą fałszywych detekcji,
  - gorszą lokalizacją obiektów.

### 2.2 Model B – model lepiej wytrenowany

Model B:
- trenowany na **lepszym jakościowo zbiorze danych**,
- trenowany przez **większą liczbę epok**,
- charakteryzuje się:
  - wyższym recall i precision,
  - lepszym dopasowaniem bounding boxów,
  - stabilniejszymi predykcjami.

## 3. Środowisko eksperymentalne

Eksperymenty przeprowadzono w środowisku:
- Python
- Jupyter Notebook
- Ultralytics YOLO
- sprzęt: Apple Silicon (dla modelu A) oraz w Chmurze obliczeniowej (dla modelu B)

Notebook pełni rolę **sprawozdania z eksperymentów** oraz **narzędzia do porównania modeli**.

## 4. Wczytanie bibliotek

In [6]:
from ultralytics import YOLO

## 5. Wczytanie wytrenowanych modeli

In [ ]:
# Model A – słabiej wytrenowany
model_poor = YOLO("GARBAGE CLASSIFICATION 3/runs/detect/train/weights/best.onnx")

# Model B – lepiej wytrenowany
model_good = YOLO("yolo11n run/runs/detect/train/weights/best.onnx")


## 6. Test modeli na wykonanym przez nas obrazie


In [8]:
image_path = "/Users/damianbrudkowski/PW/PRiAD/GarbageClassification/resources/glass.jpg"

## 7. Predykcja – Model A (słabiej wytrenowany)

In [14]:
results_poor = model_poor(image_path)
results_poor[0].show()

Loading GARBAGE CLASSIFICATION 3/runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 CPUExecutionProvider

image 1/1 /Users/damianbrudkowski/PW/PRiAD/GarbageClassification/resources/glass.jpg: 320x320 1 GLASS, 1 METAL, 7.3ms
Speed: 0.7ms preprocess, 7.3ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 320)


Obserwacje:
- model praktycznie nie wykrywa obiektów
- mozliwe błędne detekcje

## 7. Predykcja – Model B (lepiej wytrenowany)

In [15]:
results_good = model_good(image_path)
results_good[0].show()

Loading yolo11n run/runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 CPUExecutionProvider

image 1/1 /Users/damianbrudkowski/PW/PRiAD/GarbageClassification/resources/glass.jpg: 640x640 1 GLASS, 20.3ms
Speed: 2.1ms preprocess, 20.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)


Obserwacje:
- model poprawnie wykrywa obiekt
- większa pewność predykcji

## 8. Wnioski

Na podstawie przeprowadzonych eksperymentów można stwierdzić, że:

1. Jakość danych treningowych ma **kluczowy wpływ** na skuteczność modelu YOLO.
2. Zbyt krótki trening prowadzi do **niskiego recall** oraz **dużej liczby błędnych detekcji**.
3. Model trenowany dłużej na lepszych danych:
   - lepiej generalizuje,
   - zapewnia dokładniejsze i stabilniejsze predykcje.
4. Porównanie wyników uzyskanych na tym samym obrazie testowym **jednoznacznie pokazuje przewagę modelu lepiej wytrenowanego**.